### Importing all necessary libraries

In [57]:
import pandas as pd
import numpy as np
import xgboost
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error
import pickle

- Kaggle data set

In [78]:
train=pd.read_csv('train_houseprice.csv')
test=pd.read_csv('test_houseprice.csv')
pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 2)
print(train.tail())

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
1455  1456          60       RL         62.0     7917   Pave   NaN      Reg   
1456  1457          20       RL         85.0    13175   Pave   NaN      Reg   
1457  1458          70       RL         66.0     9042   Pave   NaN      Reg   
1458  1459          20       RL         68.0     9717   Pave   NaN      Reg   
1459  1460          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
1455         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
1456         Lvl    AllPub    Inside       Gtl       NWAmes       Norm   
1457         Lvl    AllPub    Inside       Gtl      Crawfor       Norm   
1458         Lvl    AllPub    Inside       Gtl        NAmes       Norm   
1459         Lvl    AllPub    Inside       Gtl      Edwards       Norm   

     Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
1455

In [79]:
test.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960.0,Unf,2.0,576.0,TA,TA,Y,474,0,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,CompShg,HdBoard,Wd Shng,None,0.0,TA,TA,PConc,Gd,TA,Av,GLQ,337.0,Unf,0.0,575.0,912.0,GasA,TA,Y,SBrkr,970,0,0,970,0.0,1.0,1,0,3,1,TA,6,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,80,32,0,0,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal
1458,2919,60,RL,74.0,9627,Pave,NaN,Reg,Lvl,AllPub,Inside,Mod,Mitchel,Norm,Norm,1Fam,2Story,7,5,1993,1994,Gable,CompShg,HdBoard,HdBoard,BrkFace,94.0,TA,TA,PConc,Gd,TA,Av,LwQ,758.0,Unf,0.0,238.0,996.0,GasA,Ex,Y,SBrkr,996,1004,0,2000,0.0,0.0,2,1,3,1,TA,9,Typ,1,TA,Attchd,1993.0,Fin,3.0,650.0,TA,TA,Y,190,48,0,0,0,0,NaN,NaN,NaN,0,11,2006,WD,Normal


Saving `saleprice` in variable `Y` and dropping `saleprice` column

In [80]:
Y=train['SalePrice'].values
train.drop('SalePrice', axis=1, inplace=True)
print(train.tail())

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
1455  1456          60       RL         62.0     7917   Pave   NaN      Reg   
1456  1457          20       RL         85.0    13175   Pave   NaN      Reg   
1457  1458          70       RL         66.0     9042   Pave   NaN      Reg   
1458  1459          20       RL         68.0     9717   Pave   NaN      Reg   
1459  1460          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
1455         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
1456         Lvl    AllPub    Inside       Gtl       NWAmes       Norm   
1457         Lvl    AllPub    Inside       Gtl      Crawfor       Norm   
1458         Lvl    AllPub    Inside       Gtl        NAmes       Norm   
1459         Lvl    AllPub    Inside       Gtl      Edwards       Norm   

     Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
1455

checking null values in training data

In [81]:
for i in range(len(train.iloc[0,:])):
    p=[i+1,train.iloc[:,i].isnull().sum()]
    print(p)

[1, 0]
[2, 0]
[3, 0]
[4, 259]
[5, 0]
[6, 0]
[7, 1369]
[8, 0]
[9, 0]
[10, 0]
[11, 0]
[12, 0]
[13, 0]
[14, 0]
[15, 0]
[16, 0]
[17, 0]
[18, 0]
[19, 0]
[20, 0]
[21, 0]
[22, 0]
[23, 0]
[24, 0]
[25, 0]
[26, 8]
[27, 8]
[28, 0]
[29, 0]
[30, 0]
[31, 37]
[32, 37]
[33, 38]
[34, 37]
[35, 0]
[36, 38]
[37, 0]
[38, 0]
[39, 0]
[40, 0]
[41, 0]
[42, 0]
[43, 1]
[44, 0]
[45, 0]
[46, 0]
[47, 0]
[48, 0]
[49, 0]
[50, 0]
[51, 0]
[52, 0]
[53, 0]
[54, 0]
[55, 0]
[56, 0]
[57, 0]
[58, 690]
[59, 81]
[60, 81]
[61, 81]
[62, 0]
[63, 0]
[64, 81]
[65, 81]
[66, 0]
[67, 0]
[68, 0]
[69, 0]
[70, 0]
[71, 0]
[72, 0]
[73, 1453]
[74, 1179]
[75, 1406]
[76, 0]
[77, 0]
[78, 0]
[79, 0]
[80, 0]


checking null values in test data 

In [82]:
for i in range(len(test.iloc[0,:])):
    p=[i+1,test.iloc[:,i].isnull().sum()]
    print(p)

[1, 0]
[2, 0]
[3, 4]
[4, 227]
[5, 0]
[6, 0]
[7, 1352]
[8, 0]
[9, 0]
[10, 2]
[11, 0]
[12, 0]
[13, 0]
[14, 0]
[15, 0]
[16, 0]
[17, 0]
[18, 0]
[19, 0]
[20, 0]
[21, 0]
[22, 0]
[23, 0]
[24, 1]
[25, 1]
[26, 16]
[27, 15]
[28, 0]
[29, 0]
[30, 0]
[31, 44]
[32, 45]
[33, 44]
[34, 42]
[35, 1]
[36, 42]
[37, 1]
[38, 1]
[39, 1]
[40, 0]
[41, 0]
[42, 0]
[43, 0]
[44, 0]
[45, 0]
[46, 0]
[47, 0]
[48, 2]
[49, 2]
[50, 0]
[51, 0]
[52, 0]
[53, 0]
[54, 1]
[55, 0]
[56, 2]
[57, 0]
[58, 730]
[59, 76]
[60, 78]
[61, 78]
[62, 1]
[63, 1]
[64, 78]
[65, 78]
[66, 0]
[67, 0]
[68, 0]
[69, 0]
[70, 0]
[71, 0]
[72, 0]
[73, 1456]
[74, 1169]
[75, 1408]
[76, 0]
[77, 0]
[78, 0]
[79, 1]
[80, 0]


concatinating train and test data 

In [83]:
data=pd.concat([train,test], axis=0, ignore_index=True)
print(data.tail())

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
2914  2915         160       RM         21.0     1936   Pave   NaN      Reg   
2915  2916         160       RM         21.0     1894   Pave   NaN      Reg   
2916  2917          20       RL        160.0    20000   Pave   NaN      Reg   
2917  2918          85       RL         62.0    10441   Pave   NaN      Reg   
2918  2919          60       RL         74.0     9627   Pave   NaN      Reg   

     LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
2914         Lvl    AllPub    Inside       Gtl      MeadowV       Norm   
2915         Lvl    AllPub    Inside       Gtl      MeadowV       Norm   
2916         Lvl    AllPub    Inside       Gtl      Mitchel       Norm   
2917         Lvl    AllPub    Inside       Gtl      Mitchel       Norm   
2918         Lvl    AllPub    Inside       Mod      Mitchel       Norm   

     Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
2914

In [84]:
for i in range(len(data.iloc[0,:])):
    p=[i+1,data.iloc[:,i].isnull().sum()]
    print(p)

[1, 0]
[2, 0]
[3, 4]
[4, 486]
[5, 0]
[6, 0]
[7, 2721]
[8, 0]
[9, 0]
[10, 2]
[11, 0]
[12, 0]
[13, 0]
[14, 0]
[15, 0]
[16, 0]
[17, 0]
[18, 0]
[19, 0]
[20, 0]
[21, 0]
[22, 0]
[23, 0]
[24, 1]
[25, 1]
[26, 24]
[27, 23]
[28, 0]
[29, 0]
[30, 0]
[31, 81]
[32, 82]
[33, 82]
[34, 79]
[35, 1]
[36, 80]
[37, 1]
[38, 1]
[39, 1]
[40, 0]
[41, 0]
[42, 0]
[43, 1]
[44, 0]
[45, 0]
[46, 0]
[47, 0]
[48, 2]
[49, 2]
[50, 0]
[51, 0]
[52, 0]
[53, 0]
[54, 1]
[55, 0]
[56, 2]
[57, 0]
[58, 1420]
[59, 157]
[60, 159]
[61, 159]
[62, 1]
[63, 1]
[64, 159]
[65, 159]
[66, 0]
[67, 0]
[68, 0]
[69, 0]
[70, 0]
[71, 0]
[72, 0]
[73, 2909]
[74, 2348]
[75, 2814]
[76, 0]
[77, 0]
[78, 0]
[79, 1]
[80, 0]


Replacing `NAN` values with `zeros`

In [85]:
loot = data['LotFrontage'].values

for i in range(len(loot)):
    # print(loot[i])
    if np.isnan(loot[i]):
       loot[i]=float(0)

loott = pd.DataFrame(loot)

data['LotFrontage'] = loott

In [86]:
Mas = data['MasVnrArea'].values

for i in range(len(Mas)):
    # print(loot[i])
    if np.isnan(Mas[i]):
       Mas[i]=float(0)

MAS = pd.DataFrame(Mas)

data['MasVnrArea'] = MAS

Bs1 = data['BsmtFinSF1'].values

for i in range(len(Bs1)):
    # print(loot[i])
    if np.isnan(Bs1[i]):
       Bs1[i]=float(0)

BS1 = pd.DataFrame(Bs1)

data['BsmtFinSF1'] = BS1

Bs2 = data['BsmtFinSF2'].values

for i in range(len(Bs2)):
    # print(loot[i])
    if np.isnan(Bs2[i]):
       Bs2[i]=float(0)

BS2 = pd.DataFrame(Bs2)

data['BsmtFinSF2'] = BS2

Bs3 = data['BsmtUnfSF'].values

for i in range(len(Bs3)):
    # print(loot[i])
    if np.isnan(Bs3[i]):
       Bs3[i]=float(0)

BS3 = pd.DataFrame(Bs3)

data['BsmtUnfSF'] = BS3

Bs4 = data['TotalBsmtSF'].values

for i in range(len(Bs4)):
    # print(loot[i])
    if np.isnan(Bs4[i]):
       Bs4[i]=float(0)

BS4 = pd.DataFrame(Bs4)

data['TotalBsmtSF'] = BS4


Bs5 = data['BsmtFullBath'].values

for i in range(len(Bs5)):
    # print(loot[i])
    if np.isnan(Bs5[i]):
       Bs5[i]=float(0)

BS5 = pd.DataFrame(Bs5)

data['BsmtFullBath'] = BS5


Bs6 = data['BsmtHalfBath'].values

for i in range(len(Bs6)):
    # print(loot[i])
    if np.isnan(Bs6[i]):
       Bs6[i]=float(0)

BS6 = pd.DataFrame(Bs6)

data['BsmtHalfBath'] = BS6

gr1 = data['GarageYrBlt'].values

for i in range(len(gr1)):
    # print(loot[i])
    if np.isnan(gr1[i]):
       gr1[i]=float(0)

g1 = pd.DataFrame(gr1)

data['GarageYrBlt'] = g1

gr2 = data['GarageArea'].values

for i in range(len(gr2)):
    # print(loot[i])
    if np.isnan(gr2[i]):
       gr2[i]=float(0)

g2 = pd.DataFrame(gr2)

data['GarageArea'] = g2

gr3 = data['GarageCars'].values

for i in range(len(gr3)):
    # print(loot[i])
    if np.isnan(gr3[i]):
       gr3[i]=float(0)

g3 = pd.DataFrame(gr3)

data['GarageCars'] = g3

replacing `NULL` values with a string 

In [87]:
Alley=data['Alley'].values
X1=data['Alley'].isnull()
print(X1)
for i in range(len(Alley)):
    if X1[i]:
        Alley[i]='No alley access'
    # print(Alley[i])

alley=pd.DataFrame(Alley)
data['Alley']=alley


0       True
1       True
2       True
3       True
4       True
        ... 
2914    True
2915    True
2916    True
2917    True
2918    True
Name: Alley, Length: 2919, dtype: bool


`one hot encoding` using `pandas.get_dummies`

In [67]:
alley_dummy=pd.get_dummies(data['Alley'])


data1=pd.concat([data,alley_dummy], axis=1)

data1.drop('Alley', axis=1,inplace=True)
print(data1.head())

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street LotShape LandContour  \
0   1          60       RL         65.0     8450   Pave      Reg         Lvl   
1   2          20       RL         80.0     9600   Pave      Reg         Lvl   
2   3          60       RL         68.0    11250   Pave      IR1         Lvl   
3   4          70       RL         60.0     9550   Pave      IR1         Lvl   
4   5          60       RL         84.0    14260   Pave      IR1         Lvl   

  Utilities LotConfig LandSlope Neighborhood Condition1 Condition2 BldgType  \
0    AllPub    Inside       Gtl      CollgCr       Norm       Norm     1Fam   
1    AllPub       FR2       Gtl      Veenker      Feedr       Norm     1Fam   
2    AllPub    Inside       Gtl      CollgCr       Norm       Norm     1Fam   
3    AllPub    Corner       Gtl      Crawfor       Norm       Norm     1Fam   
4    AllPub       FR2       Gtl      NoRidge       Norm       Norm     1Fam   

  HouseStyle  OverallQual  OverallCond  Year

In [68]:
dummyt1=pd.get_dummies(data1['MSZoning'], drop_first=True)


test2=pd.concat([data1,dummyt1], axis=1)

test2.drop('MSZoning', axis=1,inplace=True)
print(test2.head())

dummyt2=pd.get_dummies(test2['Street'], drop_first=True)


test3=pd.concat([test2,dummyt2], axis=1)

test3.drop('Street', axis=1,inplace=True)
print(test3.head())

dummyt3=pd.get_dummies(test3['LotShape'], drop_first=True)


test4=pd.concat([test3,dummyt3], axis=1)

test4.drop('LotShape', axis=1,inplace=True)
print(test4.head())


dummyt4=pd.get_dummies(test4['LandContour'], drop_first=True)


test5=pd.concat([test4,dummyt4], axis=1)

test5.drop('LandContour', axis=1,inplace=True)
print(test5.head())

dummyt5=pd.get_dummies(test5['Utilities'], drop_first=True)


test6=pd.concat([test5,dummyt5], axis=1)

test6.drop('Utilities', axis=1,inplace=True)

dummyt6=pd.get_dummies(test6['LotConfig'], drop_first=True)


test7=pd.concat([test6,dummyt6], axis=1)

test7.drop('LotConfig', axis=1,inplace=True)

dummyt7=pd.get_dummies(test7['LandSlope'], drop_first=True)


test8=pd.concat([test7,dummyt7], axis=1)

test8.drop('LandSlope', axis=1,inplace=True)

dummyt8=pd.get_dummies(test8['Neighborhood'], drop_first=True)


test9=pd.concat([test8,dummyt8], axis=1)

test9.drop('Neighborhood', axis=1,inplace=True)

dummyt9=pd.get_dummies(test9['Condition1'], drop_first=True)


test10=pd.concat([test9,dummyt9], axis=1)

test10.drop('Condition1', axis=1,inplace=True)

dummyt10=pd.get_dummies(test10['Condition2'], drop_first=True)


test11=pd.concat([test10,dummyt10], axis=1)

test11.drop('Condition2', axis=1,inplace=True)

dummyt11=pd.get_dummies(test11['BldgType'], drop_first=True)


test12=pd.concat([test11,dummyt11], axis=1)

test12.drop('BldgType', axis=1,inplace=True)
print(test12.head())

dummyt12=pd.get_dummies(test12['HouseStyle'], drop_first=True)


test13=pd.concat([test12,dummyt12], axis=1)

test13.drop('HouseStyle', axis=1,inplace=True)
print(test13.head())

dummyt13=pd.get_dummies(test13['RoofStyle'], drop_first=True)


Datat14=pd.concat([test13,dummyt13], axis=1)

Datat14.drop('RoofStyle', axis=1,inplace=True)
print(Datat14.head())

dummyt14=pd.get_dummies(Datat14['RoofMatl'], drop_first=True)


Datat15=pd.concat([Datat14,dummyt14], axis=1)

Datat15.drop('RoofMatl', axis=1,inplace=True)
print(Datat15.head())

dummyt15=pd.get_dummies(Datat15['Exterior1st'], drop_first=True)


Datat16=pd.concat([Datat15,dummyt15], axis=1)

Datat16.drop('Exterior1st', axis=1,inplace=True)
print(Datat16.head())

dummyt16=pd.get_dummies(Datat16['Exterior2nd'], drop_first=True)


Datat17=pd.concat([Datat16,dummyt16], axis=1)

Datat17.drop('Exterior2nd', axis=1,inplace=True)
print(Datat17.head())

dummyt17=pd.get_dummies(Datat17['MasVnrType'], drop_first=True)


Datat18=pd.concat([Datat17,dummyt17], axis=1)

Datat18.drop('MasVnrType', axis=1,inplace=True)
print(Datat18.head())

dummyt18=pd.get_dummies(Datat18['ExterQual'], drop_first=True)


Datat19=pd.concat([Datat18,dummyt18], axis=1)

Datat19.drop('ExterQual', axis=1,inplace=True)
print(Datat19.head())

dummyt19=pd.get_dummies(Datat19['ExterCond'], drop_first=True)


Datat20=pd.concat([Datat19,dummyt19], axis=1)

Datat20.drop('ExterCond', axis=1,inplace=True)
print(Datat20.head())

dummyt20=pd.get_dummies(Datat20['Foundation'], drop_first=True)


Datat21=pd.concat([Datat20,dummyt20], axis=1)

Datat21.drop('Foundation', axis=1,inplace=True)
print(Datat21.head())


   Id  MSSubClass  LotFrontage  LotArea Street LotShape LandContour Utilities  \
0   1          60         65.0     8450   Pave      Reg         Lvl    AllPub   
1   2          20         80.0     9600   Pave      Reg         Lvl    AllPub   
2   3          60         68.0    11250   Pave      IR1         Lvl    AllPub   
3   4          70         60.0     9550   Pave      IR1         Lvl    AllPub   
4   5          60         84.0    14260   Pave      IR1         Lvl    AllPub   

  LotConfig LandSlope Neighborhood Condition1 Condition2 BldgType HouseStyle  \
0    Inside       Gtl      CollgCr       Norm       Norm     1Fam     2Story   
1       FR2       Gtl      Veenker      Feedr       Norm     1Fam     1Story   
2    Inside       Gtl      CollgCr       Norm       Norm     1Fam     2Story   
3    Corner       Gtl      Crawfor       Norm       Norm     1Fam     2Story   
4       FR2       Gtl      NoRidge       Norm       Norm     1Fam     2Story   

   OverallQual  OverallCond  Yea

   Id  MSSubClass  LotFrontage  LotArea HouseStyle  OverallQual  OverallCond  \
0   1          60         65.0     8450     2Story            7            5   
1   2          20         80.0     9600     1Story            6            8   
2   3          60         68.0    11250     2Story            7            5   
3   4          70         60.0     9550     2Story            7            5   
4   5          60         84.0    14260     2Story            8            5   

   YearBuilt  YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd  \
0       2003          2003     Gable  CompShg     VinylSd     VinylSd   
1       1976          1976     Gable  CompShg     MetalSd     MetalSd   
2       2001          2002     Gable  CompShg     VinylSd     VinylSd   
3       1915          1970     Gable  CompShg     Wd Sdng     Wd Shng   
4       2000          2000     Gable  CompShg     VinylSd     VinylSd   

  MasVnrType  MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond  \
0  

   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd Exterior1st Exterior2nd MasVnrType  MasVnrArea ExterQual  \
0          2003     VinylSd     VinylSd    BrkFace       196.0        Gd   
1          1976     MetalSd     MetalSd       None         0.0        TA   
2          2002     VinylSd     VinylSd    BrkFace       162.0        Gd   
3          1970     Wd Sdng     Wd Shng       None         0.0        TA   
4          2000     VinylSd     VinylSd    BrkFace       350.0        Gd   

  ExterCond Foundation BsmtQual BsmtCond BsmtExposure BsmtFinT

   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond  \
0          2003       196.0        Gd        TA      PConc       Gd       TA   
1          1976         0.0        TA        TA     CBlock       Gd       TA   
2          2002       162.0        Gd        TA      PConc       Gd       TA   
3          1970         0.0        TA        TA     BrkTil       TA       Gd   
4          2000       350.0        Gd        TA      PConc       Gd       TA   

  BsmtExposure BsmtFinType1  BsmtFinSF

   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea BsmtQual BsmtCond BsmtExposure BsmtFinType1  \
0          2003       196.0       Gd       TA           No          GLQ   
1          1976         0.0       Gd       TA           Gd          ALQ   
2          2002       162.0       Gd       TA           Mn          GLQ   
3          1970         0.0       TA       Gd           No          ALQ   
4          2000       350.0       Gd       TA           Av          GLQ   

   BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF Heati

In [69]:
baset=Datat21['BsmtQual'].values
Xt1=Datat21['BsmtQual'].isnull()
print(Xt1.sum())
for i in range(len(baset)):
    if Xt1[i]:
        baset[i]='No basement'
    # print(Alley[i])

BASEt=pd.DataFrame(baset)
Datat21['BsmtQual']=BASEt

print(Datat21['BsmtQual'].isnull().sum())
dummyt21=pd.get_dummies(Datat21['BsmtQual'])


Datat22=pd.concat([Datat21,dummyt21], axis=1)

Datat22.drop('BsmtQual', axis=1,inplace=True)
print(Datat22.head())

print(Datat22['BsmtCond'].isnull().sum())
dummyt22=pd.get_dummies(Datat22['BsmtCond'], drop_first=True)


Datat23=pd.concat([Datat22,dummyt22], axis=1)

Datat23.drop('BsmtCond', axis=1,inplace=True)
print(Datat23.head())

dummyt23=pd.get_dummies(Datat23['BsmtExposure'], drop_first=True)


Datat24=pd.concat([Datat23,dummyt23], axis=1)

Datat24.drop('BsmtExposure', axis=1,inplace=True)
print(Datat24.head())

dummyt24=pd.get_dummies(Datat24['BsmtFinType1'], drop_first=True)


Datat25=pd.concat([Datat24,dummyt24], axis=1)

Datat25.drop('BsmtFinType1', axis=1,inplace=True)
print(Datat25.head())

dummyt25=pd.get_dummies(Datat25['BsmtFinType2'], drop_first=True)


Datat26=pd.concat([Datat25,dummyt25], axis=1)

Datat26.drop('BsmtFinType2', axis=1,inplace=True)
print(Datat26.head())

dummyt26=pd.get_dummies(Datat26['Heating'], drop_first=True)


Datat27=pd.concat([Datat26,dummyt26], axis=1)

Datat27.drop('Heating', axis=1,inplace=True)
print(Datat27.head())

dummyt27=pd.get_dummies(Datat27['HeatingQC'], drop_first=True)


Datat28=pd.concat([Datat27,dummyt27], axis=1)

Datat28.drop('HeatingQC', axis=1,inplace=True)
print(Datat28.head())

dummyt28=pd.get_dummies(Datat28['CentralAir'], drop_first=True)


Datat29=pd.concat([Datat28,dummyt28], axis=1)

Datat29.drop(['CentralAir'], axis=1,inplace=True)
print(Datat29.head())

print(Datat29['Electrical'].isnull().sum())

elect = Datat29['Electrical'].values
Xt3 = Datat29['Electrical'].isnull()
print(Xt3.sum())
for i in range(len(elect)):
    if Xt3[i]:
        elect[i]='No Electricity'
    # print(Alley[i])

ELECt=pd.DataFrame(elect)
Datat29['Electrical']=ELECt
dummyt29=pd.get_dummies(Datat29['Electrical'])


Datat30=pd.concat([Datat29,dummyt29], axis=1)

Datat30.drop('Electrical', axis=1,inplace=True)
print(Datat30.head())


dummyt30=pd.get_dummies(Datat30['KitchenQual'], drop_first=True)


Datat31=pd.concat([Datat30,dummyt30], axis=1)

Datat31.drop('KitchenQual', axis=1,inplace=True)
print(Datat31.head())

print(Datat31['Functional'].isnull().sum())

dummyt31=pd.get_dummies(Datat31['Functional'], drop_first=True)


Datat32=pd.concat([Datat31,dummyt31], axis=1)

Datat32.drop('Functional', axis=1,inplace=True)
print(Datat32.head())

print(Datat32['FireplaceQu'].isnull().sum())

firet = Datat32['FireplaceQu'].values
Xt4 = Datat32['FireplaceQu'].isnull()
print(Xt4.sum())
for i in range(len(firet)):
    if Xt4[i]:
        firet[i]='No Fireplaces'
    # print(Alley[i])

FIREt=pd.DataFrame(firet)
Datat32['FireplaceQu']=FIREt
dummyt32=pd.get_dummies(Datat32['FireplaceQu'])


Datat33=pd.concat([Datat32,dummyt32], axis=1)

Datat33.drop('FireplaceQu', axis=1,inplace=True)
print(Datat33.head())

garaget = Datat33['GarageType'].values
Xt5 = Datat33['GarageType'].isnull()
print(Xt5.sum())
for i in range(len(garaget)):
    if Xt5[i]:
        garaget[i]='No garage'
    # print(Alley[i])

GARAGEt=pd.DataFrame(garaget)
Datat33['GarageType']=GARAGEt
dummyt33=pd.get_dummies(Datat33['GarageType'], drop_first=True)


Datat34=pd.concat([Datat33,dummyt33], axis=1)

Datat34.drop('GarageType', axis=1,inplace=True)
print(Datat34.head())


dummyt34=pd.get_dummies(Datat34['GarageFinish'], drop_first=True)


Datat35=pd.concat([Datat34,dummyt34], axis=1)

Datat35.drop('GarageFinish', axis=1,inplace=True)
print(Datat35.tail())

dummyt35=pd.get_dummies(Datat35['GarageQual'], drop_first=True)


Datat36=pd.concat([Datat35,dummyt35], axis=1)

Datat36.drop('GarageQual', axis=1,inplace=True)
print(Datat36.head())

dummyt36=pd.get_dummies(Datat36['GarageCond'], drop_first=True)


Datat37=pd.concat([Datat36,dummyt36], axis=1)

Datat37.drop('GarageCond', axis=1,inplace=True)
print(Datat37.head())

dummyt37=pd.get_dummies(Datat37['PavedDrive'], drop_first=True)


Datat38=pd.concat([Datat37,dummyt37], axis=1)

Datat38.drop('PavedDrive', axis=1,inplace=True)
print(Datat38.head())

poolt = Datat38['PoolQC'].values
Xt6 = Datat38['PoolQC'].isnull()
print(Xt6.sum())
for i in range(len(poolt)):
    if Xt6[i]:
        poolt[i]= 'No pool'
    # print(Alley[i])

POOLt=pd.DataFrame(poolt)
Datat38['PoolQC']=POOLt

dummyt38=pd.get_dummies(Datat38['PoolQC'])


Datat39=pd.concat([Datat38,dummyt38], axis=1)

Datat39.drop('PoolQC', axis=1,inplace=True)
print(Datat39.tail())

fencet = Datat39['Fence'].values
Xt7 = Datat39['Fence'].isnull()
print(Xt7.sum())
for i in range(len(fencet)):
    if Xt7[i]:
        fencet[i]= 'No fence'
    # print(Alley[i])

FENCEt=pd.DataFrame(fencet)
Datat39['Fence']=FENCEt

dummyt39=pd.get_dummies(Datat39['Fence'])


Datat40=pd.concat([Datat39,dummyt39], axis=1)

Datat40.drop(['Fence','MiscFeature'], axis=1,inplace=True)
print(Datat40.head())

dummyt40=pd.get_dummies(Datat40['SaleType'], drop_first=True)


Datat41=pd.concat([Datat40,dummyt40], axis=1)

Datat41.drop('SaleType', axis=1,inplace=True)
print(Datat41.tail())

dummyt41=pd.get_dummies(Datat41['SaleCondition'], drop_first=True)


Datat42=pd.concat([Datat41,dummyt41], axis=1)



Datat42.drop(['SaleCondition','Id'], axis=1,inplace=True)
print(Datat42.head())


81
0
   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea BsmtCond BsmtExposure BsmtFinType1  BsmtFinSF1  \
0          2003       196.0       TA           No          GLQ       706.0   
1          1976         0.0       TA           Gd          ALQ       978.0   
2          2002       162.0       TA           Mn          GLQ       486.0   
3          1970         0.0       Gd           No          ALQ       216.0   
4          2000       350.0       TA           Av          GLQ       655.0   

  BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalB

   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea BsmtExposure BsmtFinType1  BsmtFinSF1  \
0          2003       196.0           No          GLQ       706.0   
1          1976         0.0           Gd          ALQ       978.0   
2          2002       162.0           Mn          GLQ       486.0   
3          1970         0.0           No          ALQ       216.0   
4          2000       350.0           Av          GLQ       655.0   

  BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF Heating HeatingQC  \
0          Unf         0.0      

   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  \
0          2003       196.0       706.0          Unf         0.0      150.0   
1          1976         0.0       978.0          Unf         0.0      284.0   
2          2002       162.0       486.0          Unf         0.0      434.0   
3          1970         0.0       216.0          Unf         0.0      540.0   
4          2000       350.0       655.0          Unf         0.0      490.0   

   TotalBsmtSF Heating HeatingQC CentralAir 

   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          2003       196.0       706.0         0.0      150.0        856.0   
1          1976         0.0       978.0         0.0      284.0       1262.0   
2          2002       162.0       486.0         0.0      434.0        920.0   
3          1970         0.0       216.0         0.0      540.0        756.0   
4          2000       350.0       655.0         0.0      490.0       1145.0   

  Heating HeatingQC CentralAir Electrical  1

   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          2003       196.0       706.0         0.0      150.0        856.0   
1          1976         0.0       978.0         0.0      284.0       1262.0   
2          2002       162.0       486.0         0.0      434.0        920.0   
3          1970         0.0       216.0         0.0      540.0        756.0   
4          2000       350.0       655.0         0.0      490.0       1145.0   

  CentralAir Electrical  1stFlrSF  2ndFlrSF 

...

2918       1        0        0        0       0          0    0    0  
   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          2003       196.0       706.0         0.0      150.0        856.0   
1          1976         0.0       978.0         0.0      284.0       1262.0   
2          2002       162.0       486.0         0.0      434.0        920.0   
3          1970         0.0       216.0         0.0      540.0        756.0   
4          2000       350.0       655.0         0.0  

4          0    1    0   0   0   0   1   0   0   0   1  
   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          2003       196.0       706.0         0.0      150.0        856.0   
1          1976         0.0       978.0         0.0      284.0       1262.0   
2          2002       162.0       486.0         0.0      434.0        920.0   
3          1970         0.0       216.0         0.0      540.0        756.0   
4          2000       350.0       655.0         0.0      490.0     

        Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  \
2914  2915         160         21.0     1936            4            7   
2915  2916         160         21.0     1894            4            5   
2916  2917          20        160.0    20000            5            7   
2917  2918          85         62.0    10441            5            5   
2918  2919          60         74.0     9627            7            5   

      YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  \
2914       1970          1970         0.0         0.0         0.0      546.0   
2915       1970          1970         0.0       252.0         0.0      294.0   
2916       1960          1996         0.0      1224.0         0.0        0.0   
2917       1992          1992         0.0       337.0         0.0      575.0   
2918       1993          1994        94.0       758.0         0.0      238.0   

      TotalBsmtSF  1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBa

   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          2003       196.0       706.0         0.0      150.0        856.0   
1          1976         0.0       978.0         0.0      284.0       1262.0   
2          2002       162.0       486.0         0.0      434.0        920.0   
3          1970         0.0       216.0         0.0      540.0        756.0   
4          2000       350.0       655.0         0.0      490.0       1145.0   

   1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivAr

4        0  


taking lograithm to remove skewness of data

In [70]:
numeric_features=['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
          'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
          '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
          'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
          'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
          'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']


for i in numeric_features:
    Datat42[i]=np.log(Datat42[i]+1)

print(Datat42.head())
print(Y.shape)
Y=np.log(Y+1)
print(Y)


   MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0        4.11         4.19     9.04         2.08         1.79       7.60   
1        3.04         4.39     9.17         1.95         2.20       7.59   
2        4.11         4.23     9.33         2.08         1.79       7.60   
3        4.26         4.11     9.16         2.08         1.79       7.56   
4        4.11         4.44     9.57         2.20         1.79       7.60   

   YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          7.60        5.28        6.56         0.0       5.02         6.75   
1          7.59        0.00        6.89         0.0       5.65         7.14   
2          7.60        5.09        6.19         0.0       6.08         6.83   
3          7.59        0.00        5.38         0.0       6.29         6.63   
4          7.60        5.86        6.49         0.0       6.20         7.04   

   1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHa

In [75]:
for i in range(len(Datat42.iloc[0,:])):
    p=[i+1,Datat42.iloc[:,i].isnull().sum()]
    print(p)


[1, 0]
[2, 0]
[3, 0]
[4, 0]
[5, 0]
[6, 0]
[7, 0]
[8, 0]
[9, 0]
[10, 0]
[11, 0]
[12, 0]
[13, 0]
[14, 0]
[15, 0]
[16, 0]
[17, 0]
[18, 0]
[19, 0]
[20, 0]
[21, 0]
[22, 0]
[23, 0]
[24, 0]
[25, 0]
[26, 0]
[27, 0]
[28, 0]
[29, 0]
[30, 0]
[31, 0]
[32, 0]
[33, 0]
[34, 0]
[35, 0]
[36, 0]
[37, 0]
[38, 0]
[39, 0]
[40, 0]
[41, 0]
[42, 0]
[43, 0]
[44, 0]
[45, 0]
[46, 0]
[47, 0]
[48, 0]
[49, 0]
[50, 0]
[51, 0]
[52, 0]
[53, 0]
[54, 0]
[55, 0]
[56, 0]
[57, 0]
[58, 0]
[59, 0]
[60, 0]
[61, 0]
[62, 0]
[63, 0]
[64, 0]
[65, 0]
[66, 0]
[67, 0]
[68, 0]
[69, 0]
[70, 0]
[71, 0]
[72, 0]
[73, 0]
[74, 0]
[75, 0]
[76, 0]
[77, 0]
[78, 0]
[79, 0]
[80, 0]
[81, 0]
[82, 0]
[83, 0]
[84, 0]
[85, 0]
[86, 0]
[87, 0]
[88, 0]
[89, 0]
[90, 0]
[91, 0]
[92, 0]
[93, 0]
[94, 0]
[95, 0]
[96, 0]
[97, 0]
[98, 0]
[99, 0]
[100, 0]
[101, 0]
[102, 0]
[103, 0]
[104, 0]
[105, 0]
[106, 0]
[107, 0]
[108, 0]
[109, 0]
[110, 0]
[111, 0]
[112, 0]
[113, 0]
[114, 0]
[115, 0]
[116, 0]
[117, 0]
[118, 0]
[119, 0]
[120, 0]
[121, 0]
[122, 0]
[123, 0]
[

Removing correlated features

In [88]:
def get_corr(da_ta,threshold):
    corr_col=set()
    corr_mat=da_ta.corr()
    for i in range (len(corr_mat.columns)):
        for j in range(i):
            if abs(corr_mat.iloc[i,j])>threshold:
                col_name=corr_mat.columns[i]
                corr_col.add(col_name)
    return corr_col
corr_features=get_corr(Datat42,0.95)
Data_s_c=Datat42.drop(labels=corr_features, axis=1)
print(Data_s_c.shape)
tr=Data_s_c.iloc[0:1460,:]
print(tr.shape)
te=Data_s_c.iloc[1460:2919,:]


(2919, 245)
(1460, 245)


In [89]:
scaling=StandardScaler()
data_s_tr = scaling.fit_transform(tr)
data_s_te=scaling.transform((te))
Data_s_tr=pd.DataFrame(data_s_tr)
Data_s_te=pd.DataFrame(data_s_te)

In [73]:
Parameters1=[{'reg_lambda': [0.4,0.5], 'reg_alpha': [0.9,1], 'n_estimators': [650,700], 'min_child_weight': [2.5,3],
             'max_depth': [4,3], 'learning_rate': [0.03,0.02], 'gamma': [0.00001,0.0001], 'booster': ['dart']}
               ]
scores1 = ['neg_mean_squared_error']

reg1 = GridSearchCV(xgboost.XGBRegressor(), Parameters1, scoring='neg_root_mean_squared_error', verbose=2, cv=5)
reg1.fit(Data_s_tr.iloc[:,:].values,Y)


print(reg1.best_params_)

y_pred11 = reg1.predict(Data_s_tr.iloc[:,:].values)
y_pred1 = np.exp(reg1.predict(Data_s_te.iloc[:,:].values)).round(2)
print(mean_absolute_error(y_pred11,Y))

Parameters2=[{'num_leaves':[31,32], 'max_depth':[3,4], 'learning_rate':[0.1,0.2],
            'n_estimators':[500,400]}]
scores = ['neg_mean_squared_error']

reg2 = GridSearchCV(LGBMRegressor(), Parameters2, scoring='neg_root_mean_squared_error', verbose=2, cv=5
                         )
reg2.fit(Data_s_tr.iloc[:,:].values,Y)


print(reg2.best_params_)

y_pred12 = reg2.predict(Data_s_tr.iloc[:,:].values)
y_pred2 = np.exp(reg2.predict(Data_s_te.iloc[:,:].values)).round(2)
print(mean_absolute_error(y_pred12,Y))



Parameters3=[{'cache_size': [185,180],
              'tol': [0.0011,0.0013], 'kernel': ['rbf'],'gamma': [0.00009,0.0001],'epsilon': [0.011,0.013]
              }]
scores = ['neg_mean_squared_error']

reg3 = GridSearchCV(SVR(), Parameters3, scoring='neg_root_mean_squared_error', verbose=2, cv=5
                         )
reg3.fit(Data_s_tr.iloc[:,:].values,Y)


print(reg3.best_params_)

y_pred13 = reg3.predict(Data_s_tr.iloc[:,:].values)
y_pred3 = np.exp(reg3.predict(Data_s_te.iloc[:,:].values)).round(2)
print(mean_absolute_error(y_pred13,Y))



In [ ]:
params_stack={
               'lgbmregressor__learning_rate': [0.01,0.02], 'lgbmregressor__max_depth': [3,4],
               'lgbmregressor__n_estimators': [500,600], 'lgbmregressor__num_leaves': [4,3],

              'xgbregressor__max_depth': [4,6],
              'xgbregressor__reg_lambda': [0.4,0.3],
              'xgbregressor__reg_alpha': [0.9],
              'xgbregressor__n_estimators': [500],
              'xgbregressor__min_child_weight': [2.5],
              'xgbregressor__learning_rate': [0.01,0.03],
              'xgbregressor__gamma': [0.00001],
              'xgbregressor__booster': ['dart'], 'svr__C': [75],
              'svr__cache_size': [185],
              'svr__tol': [0.0011], 'svr__kernel': ['rbf'],'svr__gamma': [0.00009],'svr__epsilon': [0.011],
              'svr__degree': [4],
              'meta_regressor__C': [75],
              'meta_regressor__cache_size': [185],
              'meta_regressor__tol': [0.0011],
              'meta_regressor__kernel': ['rbf'],
              'meta_regressor__gamma': [0.00009],
              'meta_regressor__epsilon': [0.011],
              'meta_regressor__degree': [4]
        }
xg_boost=xgboost.XGBRegressor()
s_vr=SVR()
lgbm=LGBMRegressor()

regs=[xg_boost,s_vr,lgbm]

stack_reg=StackingRegressor(regressors=regs, meta_regressor=s_vr)

stack_gen=GridSearchCV(stack_reg,params_stack,cv=5,refit=True,verbose=2)
stack_gen.fit(Data_s_tr.iloc[:,:].values,Y)
y_pred14=stack_gen.predict(Data_s_tr.iloc[:,:].values)
y_pred4=np.exp(stack_gen.predict(Data_s_te.iloc[:,:].values)).round(2)
print(stack_gen.best_params_)
print(mean_absolute_error(y_pred14,Y))


Fitting 5 folds for each of 128 candidates, totalling 640 fits
[CV] lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=3, lgbmregressor__n_estimators=500, lgbmregressor__num_leaves=4, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf, meta_regressor__tol=0.0011, svr__C=75, svr__cache_size=185, svr__degree=4, svr__epsilon=0.011, svr__gamma=9e-05, svr__kernel=rbf, svr__tol=0.0011, xgbregressor__booster=dart, xgbregressor__gamma=1e-05, xgbregressor__learning_rate=0.01, xgbregressor__max_depth=4, xgbregressor__min_child_weight=2.5, xgbregressor__n_estimators=500, xgbregressor__reg_alpha=0.9, xgbregressor__reg_lambda=0.4 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=3, lgbmregressor__n_estimators=500, lgbmregressor__num_leaves=4, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf, meta_regressor__tol=0.0011, svr__C=75, svr__cache_size=185, svr__degree=4, svr__epsilon=0.011, svr__gamma=9e-05, svr__kernel=rbf, svr__tol=0.0011, xgbregressor__booster=dart, xgbregressor__gamma=1e-05, xgbregressor__learning_rate=0.01, xgbregressor__max_depth=4, xgbregressor__min_child_weight=2.5, xgbregressor__n_estimators=500, xgbregressor__reg_alpha=0.9, xgbregressor__reg_lambda=0.4, total=  15.4s
[CV] lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=3, lgbmregressor__n_estimators=500, lgbmregressor__num_leaves=4, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf,

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.4s remaining:    0.0s


[CV]  lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=3, lgbmregressor__n_estimators=500, lgbmregressor__num_leaves=4, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf, meta_regressor__tol=0.0011, svr__C=75, svr__cache_size=185, svr__degree=4, svr__epsilon=0.011, svr__gamma=9e-05, svr__kernel=rbf, svr__tol=0.0011, xgbregressor__booster=dart, xgbregressor__gamma=1e-05, xgbregressor__learning_rate=0.01, xgbregressor__max_depth=4, xgbregressor__min_child_weight=2.5, xgbregressor__n_estimators=500, xgbregressor__reg_alpha=0.9, xgbregressor__reg_lambda=0.4, total=  13.3s
[CV] lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=3, lgbmregressor__n_estimators=500, lgbmregressor__num_leaves=4, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf,

...

[CV]  lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=4, lgbmregressor__n_estimators=500, lgbmregressor__num_leaves=3, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf, meta_regressor__tol=0.0011, svr__C=75, svr__cache_size=185, svr__degree=4, svr__epsilon=0.011, svr__gamma=9e-05, svr__kernel=rbf, svr__tol=0.0011, xgbregressor__booster=dart, xgbregressor__gamma=1e-05, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=4, xgbregressor__min_child_weight=2.5, xgbregressor__n_estimators=500, xgbregressor__reg_alpha=0.9, xgbregressor__reg_lambda=0.4, total=  14.6s
[CV] lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=4, lgbmregressor__n_estimators=500, lgbmregressor__num_leaves=3, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf,

[CV]  lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=4, lgbmregressor__n_estimators=500, lgbmregressor__num_leaves=3, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf, meta_regressor__tol=0.0011, svr__C=75, svr__cache_size=185, svr__degree=4, svr__epsilon=0.011, svr__gamma=9e-05, svr__kernel=rbf, svr__tol=0.0011, xgbregressor__booster=dart, xgbregressor__gamma=1e-05, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=4, xgbregressor__min_child_weight=2.5, xgbregressor__n_estimators=500, xgbregressor__reg_alpha=0.9, xgbregressor__reg_lambda=0.3, total=  13.8s
[CV] lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=4, lgbmregressor__n_estimators=500, lgbmregressor__num_leaves=3, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf,

[CV]  lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=4, lgbmregressor__n_estimators=500, lgbmregressor__num_leaves=3, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf, meta_regressor__tol=0.0011, svr__C=75, svr__cache_size=185, svr__degree=4, svr__epsilon=0.011, svr__gamma=9e-05, svr__kernel=rbf, svr__tol=0.0011, xgbregressor__booster=dart, xgbregressor__gamma=1e-05, xgbregressor__learning_rate=0.03, xgbregressor__max_depth=6, xgbregressor__min_child_weight=2.5, xgbregressor__n_estimators=500, xgbregressor__reg_alpha=0.9, xgbregressor__reg_lambda=0.3, total=  15.9s
[CV] lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=4, lgbmregressor__n_estimators=500, lgbmregressor__num_leaves=3, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf,

[CV]  lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=4, lgbmregressor__n_estimators=600, lgbmregressor__num_leaves=4, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf, meta_regressor__tol=0.0011, svr__C=75, svr__cache_size=185, svr__degree=4, svr__epsilon=0.011, svr__gamma=9e-05, svr__kernel=rbf, svr__tol=0.0011, xgbregressor__booster=dart, xgbregressor__gamma=1e-05, xgbregressor__learning_rate=0.01, xgbregressor__max_depth=4, xgbregressor__min_child_weight=2.5, xgbregressor__n_estimators=500, xgbregressor__reg_alpha=0.9, xgbregressor__reg_lambda=0.4, total=  14.9s
[CV] lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=4, lgbmregressor__n_estimators=600, lgbmregressor__num_leaves=4, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf,

[CV]  lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=4, lgbmregressor__n_estimators=600, lgbmregressor__num_leaves=4, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf, meta_regressor__tol=0.0011, svr__C=75, svr__cache_size=185, svr__degree=4, svr__epsilon=0.011, svr__gamma=9e-05, svr__kernel=rbf, svr__tol=0.0011, xgbregressor__booster=dart, xgbregressor__gamma=1e-05, xgbregressor__learning_rate=0.01, xgbregressor__max_depth=4, xgbregressor__min_child_weight=2.5, xgbregressor__n_estimators=500, xgbregressor__reg_alpha=0.9, xgbregressor__reg_lambda=0.3, total=  25.2s
[CV] lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=4, lgbmregressor__n_estimators=600, lgbmregressor__num_leaves=4, meta_regressor__C=75, meta_regressor__cache_size=185, meta_regressor__degree=4, meta_regressor__epsilon=0.011, meta_regressor__gamma=9e-05, meta_regressor__kernel=rbf,

In [ ]:
pred=pd.DataFrame(y_pred4)
sub_df=pd.read_csv('sample_submission.csv')
datasets=pd.concat([sub_df['Id'],pred],axis=1)
datasets.columns=['Id','SalePrice']
datasets.to_csv('sample_submission_stack.csv',index=False)